In [ ]:
#https://gis.stackexchange.com/questions/13029/converting-arcgis-server-json-to-geojson
#https://gis.stackexchange.com/questions/266897/how-to-get-around-the-1000-objectids-limit-on-arcgis-server

In [ ]:
import os
import requests
import pandas as pd
from werkzeug.utils import secure_filename
from osgeo import ogr

In [ ]:
url_feature_server = "https://ags03.sec.usace.army.mil/server/rest/services/NLD2_PUBLIC/FeatureServer/"
layer_def = requests.get(url_feature_server + '/?f=pjson').json()

df = pd.DataFrame(layer_def["layers"])
df

In [ ]:
download_folder = os.path.abspath("./download")

In [ ]:
!mkdir -p {download_folder}

In [ ]:
#id_list = [1]
id_list = range(0, df.shape[0])


def featurelayer2shapefile_ogr(featurelayer_url, shapefile_path):

    
    #ogr2ogr -overwrite -f 'ESRI Shapefile' <Shapefile> <ArcServerFeatureServer>/<LayerID>/query/?where=1+%3D+1&outfields=*&f=geojson&resultRecordCount=1000&orderByFields=OBJECTID+ASC
    
    query_url = '{}/query/?where=1+%3D+1&outfields=*&f=geojson&resultRecordCount=1000&orderByFields=OBJECTID+ASC'.format(featurelayer_url)
    print(query_url)
    ds = ogr.Open(query_url)
    layer = ds.GetLayerByIndex(0)
    feature_count = layer.GetFeatureCount()
    print("Input Feature Count: {:,}".format(feature_count))
    driver_out = ogr.GetDriverByName("ESRI Shapefile")
    # Remove output shapefile if it already exists
    if os.path.exists(shapefile_path):
        driver_out.DeleteDataSource(shapefile_path)
    ds_out = driver_out.CreateDataSource(shapefile_path)
    layer_out = ds_out.CopyLayer(layer, "layer")
    print("Output Feature Count: {:,} at {}".format(layer_out.GetFeatureCount(), shapefile_path))
    ds = None
    ds_out = None
    

def row_func(row):
    name = row["name"]    
    id = row["id"]
    if id_list is not None and type(id_list) is list:
        if int(id) not in id_list:
            return
    name_safe = secure_filename(name)
    shp_path = os.path.join(download_folder, '{}.shp'.format(name_safe))
    print("-"*80)
    print("id: {}; name: {}; file: {}".format(id, name, shp_path))
    featurelayer_url = "{}/{}/".format(url_feature_server, id)
    featurelayer2shapefile_ogr(featurelayer_url, shp_path)

In [ ]:
%%time
# the bulk download normally takes ~30 mins
_ = df.apply(row_func, axis=1)

In [ ]:
!du {download_folder} -h

In [ ]:
!ls {download_folder} -alh

## References

https://gis.stackexchange.com/questions/13029/converting-arcgis-server-json-to-geojson

https://gis.stackexchange.com/questions/266897/how-to-get-around-the-1000-objectids-limit-on-arcgis-server

https://pcjericks.github.io/py-gdalogr-cookbook/vector_layers.html#get-wfs-layers-and-iterate-over-features

https://gdal.org/python/

https://gdal.org/drivers/vector/geojson.html#vector-geojson

https://gdal.org/drivers/vector/esrijson.html#vector-esrijson